In [1]:
import sys
#sys.path.append('/home/mdd424/CARL_tthbb/')
#sys.path.append('/home/mdd424/downloads/carl-torch')

import uproot
import numpy as np
import math
import json
import bisect
import os
import pickle
import logging

import torch
from torch import nn
from torch import sigmoid
from torch.utils.data import DataLoader
#from torchsummary import summary

from sklearn import preprocessing
from sklearn.isotonic import IsotonicRegression
from sklearn.calibration import calibration_curve
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KernelDensity

from scipy.spatial import distance
import awkward as ak

import matplotlib as mpl
default_backend = mpl.get_backend()
print(default_backend)
import matplotlib.pyplot as plt
#import imageio

module://matplotlib_inline.backend_inline


In [2]:
mpl.use(default_backend)

In [3]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cpu')

In [4]:
jet_features = ["Jet_pt", "Jet_eta", "Jet_phi", "Jet_mass"]
electron_features = ["Electron_pt", "Electron_eta", "Electron_phi", "Electron_mass"]
muon_features = ["Muon_pt", "Muon_eta", "Muon_phi", "Muon_mass"]

In [5]:
#weight_features = ["weight_mc", "weight_pileup", "weight_leptonSF", "weight_jvt", "weight_bTagSF_DL1r_Continuous"]
weight_features = ["genWeight", "btagWeight_CSVV2"]

In [6]:
# Luminosities in pb^-1
luminosities = {'2015': 36207.66, '2017': 44307.4, '2018': 58450.1}
luminosities_by_run = {'9364': 36207.66, '10201': 44307.4, '10724': 58450.1}

In [7]:
with open("nanoaod_inputs.json", 'r') as f:
    file_dict = json.load(f)

In [8]:
data_metadata_dict = {}

In [9]:
all_nominal_files = [x["path"] for x in file_dict["ttbar"]["nominal"]["files"]]
print(len(all_nominal_files), all_nominal_files[0])
np.random.shuffle(all_nominal_files)
all_nominal_files[0]

243 https://xrootd-local.unl.edu:1094//store/user/AGC/nanoAOD/TT_TuneCUETP8M1_13TeV-powheg-pythia8/cmsopendata2015_ttbar_19980_PU25nsData2015v1_76X_mcRun2_asymptotic_v12_ext3-v1_00000_0000.root


'https://xrootd-local.unl.edu:1094//store/user/AGC/nanoAOD/TT_TuneCUETP8M1_13TeV-powheg-pythia8/cmsopendata2015_ttbar_19981_PU25nsData2015v1_76X_mcRun2_asymptotic_v12_ext4-v1_60001_0009.root'

In [10]:
all_variation_files = [x["path"] for x in file_dict["ttbar"]["PS_var"]["files"]]
print(len(all_variation_files), all_variation_files[0])
np.random.shuffle(all_variation_files)
all_variation_files[0]

15 https://xrootd-local.unl.edu:1094//store/user/AGC/nanoAOD/TT_TuneEE5C_13TeV-powheg-herwigpp/cmsopendata2015_ttbar_19999_PU25nsData2015v1_76X_mcRun2_asymptotic_v12-v1_10000_0000.root


'https://xrootd-local.unl.edu:1094//store/user/AGC/nanoAOD/TT_TuneEE5C_13TeV-powheg-herwigpp/cmsopendata2015_ttbar_19999_PU25nsData2015v1_76X_mcRun2_asymptotic_v12-v1_10000_0011.root'

In [11]:
"""nominal_Nevents = 0
variation_Nevents = 0
for filename in all_nominal_files:
    dataset = uproot.open(filename)["Events"].arrays(jet_features[0])
    nominal_Nevents += len(dataset)
for filename in all_variation_files:
    dataset = uproot.open(filename)["Events"].arrays(jet_features[0])
    variation_Nevents += len(dataset)
max_data_index = min([nominal_Nevents, variation_Nevents])
max_data_index"""

'nominal_Nevents = 0\nvariation_Nevents = 0\nfor filename in all_nominal_files:\n    dataset = uproot.open(filename)["Events"].arrays(jet_features[0])\n    nominal_Nevents += len(dataset)\nfor filename in all_variation_files:\n    dataset = uproot.open(filename)["Events"].arrays(jet_features[0])\n    variation_Nevents += len(dataset)\nmax_data_index = min([nominal_Nevents, variation_Nevents])\nmax_data_index'

In [12]:
with open("carl_data_metadata.json", 'r') as f:
    data_metadata_dict = json.load(f)
    max_data_index = data_metadata_dict["max_data_index"]
    max_jet_size = data_metadata_dict["max_data_index"]
    max_electron_size = data_metadata_dict["max_electron_size"]
    max_muon_size = data_metadata_dict["max_muon_size"]

In [13]:
train_frac = 0.6
val_frac = 0.2
test_frac = 0.2

max_train_index = int(train_frac * max_data_index)
max_val_index = max_train_index + int(val_frac * max_data_index)
max_test_index = max_val_index + int(test_frac * max_data_index)

np.random.shuffle(all_nominal_files)
np.random.shuffle(all_variation_files)
print(max_train_index, max_val_index, max_test_index)

11602238 15469650 19337062


In [14]:
def get_max_sizes(filename):
    dataset = uproot.open(filename)["Events"].arrays([jet_features[0], electron_features[0], muon_features[0]])
    max_jet_size = max(map(len, dataset[jet_features[0]]))
    max_electron_size = max(map(len, dataset[electron_features[0]]))
    max_muon_size = max(map(len, dataset[muon_features[0]]))
    return [max_jet_size, max_electron_size, max_muon_size]

In [15]:
#max_jet_size = 0
#for filename in all_nominal_files + all_variation_files:
#    dataset = uproot.open(filename)["Events"].arrays(jet_features)
#    max_jet_size = max([max(map(len, dataset[jet_features[0]])), max_jet_size]) 

In [16]:
#max_jet_size = 20
#max_jet_size

## Generate the nominal datasets

In [17]:
def fill_or_extend_tree(datafile, tree_dict, treename="Events"):
    if datafile.get(treename) is None:
        datafile[treename] = tree_dict
    else:
        datafile[treename].extend(tree_dict)
    return None

In [18]:
def build_data_dict(features, arrays, split_index=None, split_low=False, split_high=False):
    if split_low is False and split_high is False:
        data_dict = dict(zip(features, ak.unzip(arrays)))
    elif split_high is True:
        data_dict = dict(zip(features, ak.unzip(arrays[:split_index])))
    elif split_low is True:
        data_dict = dict(zip(features, ak.unzip(arrays[split_index:])))
    return data_dict

In [ ]:
df_train = uproot.recreate("data/CMS_ttbar_nominal_training_data.root")
df_val = uproot.recreate("data/CMS_ttbar_nominal_validation_data.root")
df_test = uproot.recreate("data/CMS_ttbar_nominal_testing_data.root")

chunk_size = 500

current_index = 0
for filename in all_nominal_files:
    print("Loading file: {}".format(filename))
    # Load the data
    nominal_dataset = uproot.open(filename)["Events"]#.arrays(jet_features + electron_features + muon_features + weight_features)
    filesize = int(nominal_dataset.arrays(jet_features[0]).type.length)
    print(filesize)
    for i in range(int(np.ceil(filesize / chunk_size))):
        #data_arrays = nominal_dataset.arrays(jet_features + electron_features + muon_features + weight_features,
        #                                     entry_start=int(i * chunk_size),
        #                                     entry_stop=int((i+1) * chunk_size))

        jet_arr = nominal_dataset.arrays(jet_features, entry_start=int(i * chunk_size), entry_stop=int((i+1) * chunk_size))
        electron_arr = nominal_dataset.arrays(electron_features, entry_start=int(i * chunk_size), entry_stop=int((i+1) * chunk_size))
        muon_arr = nominal_dataset.arrays(muon_features, entry_start=int(i * chunk_size), entry_stop=int((i+1) * chunk_size))
        weight_arr = nominal_dataset.arrays(weight_features, entry_start=int(i * chunk_size), entry_stop=int((i+1) * chunk_size))
        # Get the run number
        #nominal_file_run_number = str(NOMINAL_FILE_TO_RUN_NUMBER[filename])
        # Get the DSID number
        #nominal_file_dsid = str(NOMINAL_FILE_TO_DSID[filename])
        # Get the luminsotiy, DSID cross section, and per DSID total weighted events
        #_scale_factor = luminosities_by_run[nominal_file_run_number] * NOMINAL_XSECTIONS[nominal_file_dsid] / NOMINAL_NORMALIZATIONS[nominal_file_dsid]
        _scale_factor = 3378 * 1 / 1
        # Extract the combined weight array
        _weights = ak.concatenate(ak.unzip(weight_arr[weight_features][:, np.newaxis]), axis=1).to_numpy().prod(axis=1)

        nominal_padded_jets = ak.fill_none(ak.pad_none(jet_arr[jet_features], max_jet_size, clip=True), 0, axis=1)
        nominal_padded_electrons = ak.fill_none(ak.pad_none(electron_arr[electron_features], max_electron_size, clip=True), 0, axis=1)
        nominal_padded_muons = ak.fill_none(ak.pad_none(muon_arr[muon_features], max_muon_size, clip=True), 0, axis=1)

        current_data_size = len(_weights)

        print("Here")
        # put everything in train
        if current_index + current_data_size < max_train_index:
            #data_dict = dict(zip(jet_features, ak.unzip(nominal_padded_jets)))
            data_dict = build_data_dict(jet_features, nominal_padded_jets)
            data_dict.update(build_data_dict(electron_features, nominal_padded_electrons))
            data_dict.update(build_data_dict(muon_features, nominal_padded_muons))
            data_dict["weight_mc_combined"] = _weights * _scale_factor
            fill_or_extend_tree(df_train, data_dict)
        # put part in train and the rest in val
        elif current_index < max_train_index and current_index + current_data_size < max_val_index:
            split_index = max_train_index - current_index
            data_dict = build_data_dict(jet_features, nominal_padded_jets, split_index=split_index, split_high=True)
            data_dict.update(build_data_dict(electron_features, nominal_padded_electrons, split_index=split_index, split_high=True))
            data_dict.update(build_data_dict(muon_features, nominal_padded_muons, split_index=split_index, split_high=True))
            data_dict["weight_mc_combined"] = _weights[:split_index] * _scale_factor
            fill_or_extend_tree(df_train, data_dict)

            data_dict = build_data_dict(jet_features, nominal_padded_jets, split_index=split_index, split_low=True)
            data_dict.update(build_data_dict(electron_features, nominal_padded_electrons, split_index=split_index, split_low=True))
            data_dict.update(build_data_dict(muon_features, nominal_padded_muons, split_index=split_index, split_low=True))
            data_dict["weight_mc_combined"] = _weights[split_index:] * _scale_factor
            fill_or_extend_tree(df_val, data_dict)
        # put everything into val
        elif current_index >= max_train_index and current_index + current_data_size < max_val_index:
            data_dict = build_data_dict(jet_features, nominal_padded_jets)
            data_dict.update(build_data_dict(electron_features, nominal_padded_electrons))
            data_dict.update(build_data_dict(muon_features, nominal_padded_muons))
            data_dict["weight_mc_combined"] = _weights * _scale_factor
            fill_or_extend_tree(df_val, data_dict)
        # put part in val and the rest in test
        elif current_index < max_val_index and current_index + current_data_size < max_test_index:
            split_index = max_val_index - current_index
            data_dict = build_data_dict(jet_features, nominal_padded_jets, split_index=split_index, split_high=True)
            data_dict.update(build_data_dict(electron_features, nominal_padded_electrons, split_index=split_index, split_high=True))
            data_dict.update(build_data_dict(muon_features, nominal_padded_muons, split_index=split_index, split_high=True))
            data_dict["weight_mc_combined"] = _weights[:split_index] * _scale_factor
            fill_or_extend_tree(df_val, data_dict)

            data_dict = build_data_dict(jet_features, nominal_padded_jets, split_index=split_index, split_low=True)
            data_dict.update(build_data_dict(electron_features, nominal_padded_electrons, split_index=split_index, split_low=True))
            data_dict.update(build_data_dict(muon_features, nominal_padded_muons, split_index=split_index, split_low=True))
            data_dict["weight_mc_combined"] = _weights[split_index:] * _scale_factor
            fill_or_extend_tree(df_test, data_dict)
        # put everything into test
        elif current_index >= max_val_index and current_index + current_data_size < max_test_index:
            data_dict = build_data_dict(jet_features, nominal_padded_jets)
            data_dict.update(build_data_dict(electron_features, nominal_padded_electrons))
            data_dict.update(build_data_dict(muon_features, nominal_padded_muons))
            data_dict["weight_mc_combined"] = _weights * _scale_factor
            fill_or_extend_tree(df_test, data_dict)
        # put what's needed into test and ignore the rest
        elif current_index < max_test_index and current_index + current_data_size >= max_test_index:
            split_index = max_test_index - current_index
            data_dict = build_data_dict(jet_features, nominal_padded_jets, split_index=split_index, split_high=True)
            data_dict.update(build_data_dict(electron_features, nominal_padded_electrons, split_index=split_index, split_high=True))
            data_dict.update(build_data_dict(muon_features, nominal_padded_muons, split_index=split_index, split_high=True))
            data_dict["weight_mc_combined"] = _weights[:split_index] * _scale_factor
            fill_or_extend_tree(df_test, data_dict)
        else:
            print("Uncaught case:",
                  "current_index: {}".format(current_index),
                  "current_data_size: {}".format(current_data_size),
                  "max_train_index: {}".format(max_train_index),
                  "max_val_index: {}".format(max_val_index),
                  "max_test_index: {}".format(max_test_index), sep='\n')

        current_index += current_data_size
        if current_index >= max_test_index:
            break
print("Finished!")

Loading file: https://xrootd-local.unl.edu:1094//store/user/AGC/nanoAOD/TT_TuneCUETP8M1_13TeV-powheg-pythia8/cmsopendata2015_ttbar_19981_PU25nsData2015v1_76X_mcRun2_asymptotic_v12_ext4-v1_40001_0001.root
1225600


In [18]:
df_train = uproot.recreate("mixture_model_data/Sherpa_ttbb_full_training_data.root")
df_val = uproot.recreate("mixture_model_data/Sherpa_ttbb_full_validation_data.root")
df_test = uproot.recreate("mixture_model_data/Sherpa_ttbb_full_testing_data.root")

current_index = 0
for filename in all_variation_files:
    print("Loading file: {}".format(filename))
    # Load the data
    variation_dataset = uproot.open(filename)["nominal_Loose"].arrays(jet_features + weight_features)
    # Get the run number
    variation_file_run_number = str(VARIATION_FILE_TO_RUN_NUMBER[filename])
    # Get the DSID number
    variation_file_dsid = str(VARIATION_FILE_TO_DSID[filename])
    # Get the luminsotiy, DSID cross section, and per DSID total weighted events
    _scale_factor = luminosities_by_run[variation_file_run_number] * VARIATION_XSECTIONS[variation_file_dsid] / VARIATION_NORMALIZATIONS[variation_file_dsid]
    # Extract the combined weight array
    _weights = ak.concatenate(ak.unzip(variation_dataset[weight_features][:, np.newaxis]), axis=1).to_numpy().prod(axis=1)
    
    variation_padded_jets = ak.fill_none(ak.pad_none(variation_dataset[jet_features], max_jet_size, clip=True), 0, axis=1)
    
    current_data_size = len(_weights)
    
    # put everything in train
    if current_index + current_data_size < max_train_index:
        data_dict = dict(zip(jet_features, ak.unzip(variation_padded_jets)))
        data_dict["weight_mc_combined"] = _weights * _scale_factor
        fill_or_extend_tree(df_train, data_dict)
    # put part in train and the rest in val
    elif current_index < max_train_index and current_index + current_data_size < max_val_index:
        split_index = max_train_index - current_index
        data_dict = dict(zip(jet_features, ak.unzip(variation_padded_jets[:split_index])))
        data_dict["weight_mc_combined"] = _weights[:split_index] * _scale_factor
        fill_or_extend_tree(df_train, data_dict)
        
        data_dict = dict(zip(jet_features, ak.unzip(variation_padded_jets[split_index:])))
        data_dict["weight_mc_combined"] = _weights[split_index:] * _scale_factor
        fill_or_extend_tree(df_val, data_dict)
    # put everything into val
    elif current_index >= max_train_index and current_index + current_data_size < max_val_index:
        data_dict = dict(zip(jet_features, ak.unzip(variation_padded_jets)))
        data_dict["weight_mc_combined"] = _weights * _scale_factor
        fill_or_extend_tree(df_val, data_dict)
    # put part in val and the rest in test
    elif current_index < max_val_index and current_index + current_data_size < max_test_index:
        split_index = max_val_index - current_index
        data_dict = dict(zip(jet_features, ak.unzip(variation_padded_jets[:split_index])))
        data_dict["weight_mc_combined"] = _weights[:split_index] * _scale_factor
        fill_or_extend_tree(df_val, data_dict)
        
        data_dict = dict(zip(jet_features, ak.unzip(variation_padded_jets[split_index:])))
        data_dict["weight_mc_combined"] = _weights[split_index:] * _scale_factor
        fill_or_extend_tree(df_test, data_dict)
    # put everything into test
    elif current_index >= max_val_index and current_index + current_data_size < max_test_index:
        data_dict = dict(zip(jet_features, ak.unzip(variation_padded_jets)))
        data_dict["weight_mc_combined"] = _weights * _scale_factor
        fill_or_extend_tree(df_test, data_dict)
    # put what's needed into test and ignore the rest
    elif current_index < max_test_index and current_index + current_data_size >= max_test_index:
        split_index = max_test_index - current_index
        data_dict = dict(zip(jet_features, ak.unzip(variation_padded_jets[:split_index])))
        data_dict["weight_mc_combined"] = _weights[:split_index] * _scale_factor
        fill_or_extend_tree(df_test, data_dict)
    else:
        print("Uncaught case:",
              "current_index: {}".format(current_index),
              "current_data_size: {}".format(current_data_size),
              "max_train_index: {}".format(max_train_index),
              "max_val_index: {}".format(max_val_index),
              "max_test_index: {}".format(max_test_index), sep='\n')
    
    current_index += current_data_size
    if current_index >= max_test_index:
        break
print("Finished!")

Loading file: /scratch/mdd424/CARL_tthbb/data/Sherpa/semilep/user.alheld.mc16_13TeV.700165.Sh_2210_ttbb_ljetsP.TOPQ1.e8263a875r10724p4434.TTHbb212206-v1n_1l_out_root/user.alheld.28403455._000023.1l_out.root
Loading file: /scratch/mdd424/CARL_tthbb/data/Sherpa/semilep/user.alheld.mc16_13TeV.700166.Sh_2210_ttbbljetsM.TOPQ1.e8263a875r10201p4434.TTHbb212206-v1n_1l_out_root/user.alheld.28403761._000039.1l_out.root
Loading file: /scratch/mdd424/CARL_tthbb/data/Sherpa/semilep/user.alheld.mc16_13TeV.700166.Sh_2210_ttbbljetsM.TOPQ1.e8263a875r10724p4434.TTHbb212206-v1n_1l_out_root/user.alheld.28403457._000023.1l_out.root
Loading file: /scratch/mdd424/CARL_tthbb/data/Sherpa/semilep/user.alheld.mc16_13TeV.700166.Sh_2210_ttbbljetsM.TOPQ1.e8263a875r10201p4434.TTHbb212206-v1n_1l_out_root/user.alheld.28403761._000007.1l_out.root
Loading file: /scratch/mdd424/CARL_tthbb/data/Sherpa/semilep/user.alheld.mc16_13TeV.700166.Sh_2210_ttbbljetsM.TOPQ1.e8263a875r10724p4434.TTHbb212206-v1n_1l_out_root/user.alhel

# ------------------------------
# Section 2
# ------------------------------
# Create a dataset using the weight estimators as replacements for the MC weights

In [9]:
class WeightRegression(nn.Module):
    def __init__(self, input_dim, num_layers=3, nodes_per_layer=50, x_scaler=None, y_scaler=None, bandwidth=1):
        super(WeightRegression, self).__init__()
        
        self._x_scaler = x_scaler
        self._y_scaler = y_scaler
        self._bandwidth = bandwidth
        
        self._flatten = nn.Flatten()
        relu_layers = []
        relu_layers.append(nn.Linear(input_dim, nodes_per_layer))
        relu_layers.append(nn.ReLU())
        for i in range(num_layers-1):
            relu_layers.append(nn.Linear(nodes_per_layer, nodes_per_layer))
            relu_layers.append(nn.ReLU())
        relu_layers.append(nn.Linear(nodes_per_layer, 1)) # output layer -> dim=1 for weight prediction
        
        self._relu_layers = nn.Sequential(*relu_layers)
        
        self._loss = nn.MSELoss(reduction="none")
        

    def scale_x(self, x, inverse=False):
        if self._x_scaler is None:
            return x
        else:
            if inverse is False:
                return self._x_scaler.transform(x)
            else:
                return self._x_scaler.inverse_transform(x)


    def scale_y(self, y, inverse=False):
        if self._y_scaler is None:
            return y
        else:
            if inverse is False:
                return self._y_scaler.transform(y)
            else:
                return self._y_scaler.inverse_transform(y)
        
        
    def forward(self, x):
        x = self._flatten(x)
        x = self._relu_layers(x)
        return x

    
    def kernel(self, x, y, bandwidth=1):
        return torch.exp(-torch.pow(x-y, 2)/(2*bandwidth))   
        #return torch.exp(-LA.vector_norm(x-y, dim=1)/(2*self._bandwidth))        
    
    
    def get_MMD(self, x, pred, target):
        N = len(target)
        #pred = pred.reshape(-1,1)
        #target = target.reshape(-1,1)
        nn_x = x*pred
        mc_x = x*target
        
        term1 = torch.zeros(1, requires_grad=True)
        term2 = torch.zeros(1,requires_grad=True)
        term3 = torch.zeros(1, requires_grad=True)
        
        bandwidths = [0.5, 1, 2, 5, 10]
        for i in range(N):
            for j in range(N):
                for b in bandwidths:
                    term1 = term1 + self.kernel(nn_x[i], nn_x[j], bandwidth=b)
                    term2 = term2 + self.kernel(nn_x[i], mc_x[j], bandwidth=b)
                    term3 = term3 + self.kernel(mc_x[i], mc_x[j], bandwidth=b)
        loss = (term1 + term3 - 2*term2) / N**2
        return loss
    
    
    def get_loss(self, x, pred, target, weights=None):
        N = len(target)
        #loss = torch.pow(torch.mean(x*pred)- torch.mean(x*target), 2)
        #loss = loss + torch.pow(torch.mean(torch.pow(x*pred, 2)) - torch.mean(torch.pow(x*target, 2)), 2)/2
        #loss = loss + torch.pow(torch.mean(torch.pow(x*pred, 3)) - torch.mean(torch.pow(x*target, 3)), 2)/6
        #return loss
        if weights is None:
            return self._loss(pred, target).mean()
        else:
            return (self._loss(pred, target)*weights).mean()

## Load the scaling metadata for these models

In [11]:
with open("PP8_ttbb_weights_jets_model_metadata.json", 'r') as f:
    PP8_weights_model_metadata = json.load(f)
    PP8_weights_model_X_scaler = preprocessing.StandardScaler()
    PP8_weights_model_y_scaler = preprocessing.StandardScaler()
    
    PP8_weights_model_X_scaler.scale_ = PP8_weights_model_metadata["x scale"]["scale"]
    PP8_weights_model_X_scaler.mean_ = PP8_weights_model_metadata["x scale"]["mean"]
    PP8_weights_model_X_scaler.var_ = PP8_weights_model_metadata["x scale"]["var"]
    PP8_weights_model_X_scaler.n_features_in_ = PP8_weights_model_metadata["x scale"]["n_features_in"]
    
    PP8_weights_model_y_scaler.scale_ = PP8_weights_model_metadata["weight scale"]["scale"]
    PP8_weights_model_y_scaler.mean_ = PP8_weights_model_metadata["weight scale"]["mean"]
    PP8_weights_model_y_scaler.var_ = PP8_weights_model_metadata["weight scale"]["var"]
    PP8_weights_model_y_scaler.n_features_in_ = PP8_weights_model_metadata["weight scale"]["n_features_in"]
    
with open("Sherpa_ttbb_weights_jets_model_metadata.json", 'r') as f:
    Sherpa_weights_model_metadata = json.load(f)
    Sherpa_weights_model_X_scaler = preprocessing.StandardScaler()
    Sherpa_weights_model_y_scaler = preprocessing.StandardScaler()
    
    Sherpa_weights_model_X_scaler.scale_ = Sherpa_weights_model_metadata["x scale"]["scale"]
    Sherpa_weights_model_X_scaler.mean_ = Sherpa_weights_model_metadata["x scale"]["mean"]
    Sherpa_weights_model_X_scaler.var_ = Sherpa_weights_model_metadata["x scale"]["var"]
    Sherpa_weights_model_X_scaler.n_features_in_ = Sherpa_weights_model_metadata["x scale"]["n_features_in"]
    
    Sherpa_weights_model_y_scaler.scale_ = Sherpa_weights_model_metadata["weight scale"]["scale"]
    Sherpa_weights_model_y_scaler.mean_ = Sherpa_weights_model_metadata["weight scale"]["mean"]
    Sherpa_weights_model_y_scaler.var_ = Sherpa_weights_model_metadata["weight scale"]["var"]
    Sherpa_weights_model_y_scaler.n_features_in_ = Sherpa_weights_model_metadata["weight scale"]["n_features_in"]

## Load the weight models

In [13]:
PP8_weights_model = WeightRegression(len(PP8_weights_model_metadata["features"]),
                                     num_layers=PP8_weights_model_metadata["num_layers"],
                                     nodes_per_layer=PP8_weights_model_metadata["nodes_per_layer"])
PP8_weights_model.load_state_dict(torch.load("PP8_ttbb_weights_jets_model.pt"))
PP8_weights_model.eval()

Sherpa_weights_model = WeightRegression(len(Sherpa_weights_model_metadata["features"]),
                                        num_layers=Sherpa_weights_model_metadata["num_layers"],
                                        nodes_per_layer=Sherpa_weights_model_metadata["nodes_per_layer"])
Sherpa_weights_model.load_state_dict(torch.load("Sherpa_ttbb_weights_jets_model.pt"))
Sherpa_weights_model.eval()

WeightRegression(
  (_flatten): Flatten(start_dim=1, end_dim=-1)
  (_relu_layers): Sequential(
    (0): Linear(in_features=80, out_features=200, bias=True)
    (1): ReLU()
    (2): Linear(in_features=200, out_features=200, bias=True)
    (3): ReLU()
    (4): Linear(in_features=200, out_features=200, bias=True)
    (5): ReLU()
    (6): Linear(in_features=200, out_features=200, bias=True)
    (7): ReLU()
    (8): Linear(in_features=200, out_features=1, bias=True)
  )
  (_loss): MSELoss()
)

## Evaluate the models on the training data to get new weights

In [16]:
nominal_tree = uproot.open("mixture_model_data/PP8_ttbb_full_validation_data.root")["nominal_Loose"]
alternative_tree = uproot.open("mixture_model_data/Sherpa_ttbb_full_validation_data.root")["nominal_Loose"]

X_total_datasets = [ak.concatenate(ak.unzip(nominal_tree.arrays(jet_features)), axis=1).to_numpy().astype(np.float32),
                    ak.concatenate(ak.unzip(alternative_tree.arrays(jet_features)), axis=1).to_numpy().astype(np.float32)]

PP8_X_input = torch.from_numpy(PP8_weights_model_X_scaler.transform(X_total_datasets[0]))
Sherpa_X_input = torch.from_numpy(Sherpa_weights_model_X_scaler.transform(X_total_datasets[1]))

with torch.no_grad():
    PP8_pred_vals = PP8_weights_model(PP8_X_input)
    Sherpa_pred_vals = Sherpa_weights_model(Sherpa_X_input)

PP8_nn_weights = PP8_weights_model_y_scaler.inverse_transform(PP8_pred_vals.reshape(-1,1))
Sherpa_nn_weights = Sherpa_weights_model_y_scaler.inverse_transform(Sherpa_pred_vals.reshape(-1,1))

In [17]:
PP8_nn_weights.mean() / PP8_weights_model_metadata["weight scale"]["mean"], Sherpa_nn_weights.mean() / Sherpa_weights_model_metadata["weight scale"]["mean"]

(array([1.00167877]), array([1.0033104]))

## Save the new training data with the replacement weights

In [21]:
df = uproot.recreate("mixture_model_data/PP8_ttbb_train_jets_weight_classifier.root")

weight_array = PP8_nn_weights.ravel()
# Normalize by the mean for training stability
weight_array /= PP8_weights_model_metadata["weight scale"]["mean"]

data_dict = dict()
for ix, feat in enumerate(PP8_weights_model_metadata["features"]):
    data_dict[feat] = X_total_datasets[0][:, ix].ravel()
data_dict["weight_mc_combined"] = weight_array

df["nominal_Loose"] = data_dict

In [22]:
df = uproot.recreate("mixture_model_data/Sherpa_ttbb_train_jets_weight_classifier.root")

weight_array = Sherpa_nn_weights.ravel()
# Normalize by the mean for training stability
weight_array /= Sherpa_weights_model_metadata["weight scale"]["mean"]

data_dict = dict()
for ix, feat in enumerate(Sherpa_weights_model_metadata["features"]):
    data_dict[feat] = X_total_datasets[1][:, ix].ravel()
data_dict["weight_mc_combined"] = weight_array

df["nominal_Loose"] = data_dict

# ============================
# Section 3: Make a Deep Sets training dataset
# ============================

In [147]:
df_train = uproot.recreate("mixture_model_data/PP8_ttbb_full_training_DeepSets_data.root")
df_val = uproot.recreate("mixture_model_data/PP8_ttbb_full_validation_DeepSets_data.root")
df_test = uproot.recreate("mixture_model_data/PP8_ttbb_full_testing_DeepSets_data.root")

current_index = 0
for filename in all_nominal_files:
    print("Loading file: {}".format(filename))
    # Load the data
    nominal_dataset = uproot.open(filename)["nominal_Loose"].arrays(jet_features + weight_features)
    # Get the run number
    nominal_file_run_number = str(NOMINAL_FILE_TO_RUN_NUMBER[filename])
    # Get the DSID number
    nominal_file_dsid = str(NOMINAL_FILE_TO_DSID[filename])
    # Get the luminsotiy, DSID cross section, and per DSID total weighted events
    _scale_factor = luminosities_by_run[nominal_file_run_number] * NOMINAL_XSECTIONS[nominal_file_dsid] / NOMINAL_NORMALIZATIONS[nominal_file_dsid]
    # Extract the combined weight array
    _weights = ak.concatenate(ak.unzip(nominal_dataset[weight_features][:, np.newaxis]), axis=1).to_numpy().prod(axis=1)
    
    current_data_size = len(_weights)
    
    jet_sets = []
    for i in range(current_data_size):
        jet_sets.append(
            np.concatenate([x.to_numpy()[:, np.newaxis] for x in ak.unzip(nominal_dataset[jet_features][i])], axis=1).flatten()
        )
    jet_arr = ak.Array(jet_sets)
    
    # put everything in train
    if current_index + current_data_size < max_train_index:
        data_dict = {"jet_4vec": jet_arr}
        data_dict["weight_mc_combined"] = _weights * _scale_factor
        fill_or_extend_tree(df_train, data_dict)
    # put part in train and the rest in val
    elif current_index < max_train_index and current_index + current_data_size < max_val_index:
        split_index = max_train_index - current_index
        data_dict = {"jet_4vec": jet_arr[:split_index]}
        data_dict["weight_mc_combined"] = _weights[:split_index] * _scale_factor
        fill_or_extend_tree(df_train, data_dict)
        
        data_dict = {"jet_4vec": jet_arr[split_index:]}
        data_dict["weight_mc_combined"] = _weights[split_index:] * _scale_factor
        fill_or_extend_tree(df_val, data_dict)
    # put everything into val
    elif current_index >= max_train_index and current_index + current_data_size < max_val_index:
        data_dict = {"jet_4vec": jet_arr}
        data_dict["weight_mc_combined"] = _weights * _scale_factor
        fill_or_extend_tree(df_val, data_dict)
    # put part in val and the rest in test
    elif current_index < max_val_index and current_index + current_data_size < max_test_index:
        split_index = max_val_index - current_index
        data_dict = {"jet_4vec": jet_arr[:split_index]}
        data_dict["weight_mc_combined"] = _weights[:split_index] * _scale_factor
        #fill_or_extend_tree(df_val, data_dict)
        
        data_dict = {"jet_4vec": jet_arr[split_index:]}
        data_dict["weight_mc_combined"] = _weights[split_index:] * _scale_factor
        fill_or_extend_tree(df_test, data_dict)
    # put everything into test
    elif current_index >= max_val_index and current_index + current_data_size < max_test_index:
        data_dict = {"jet_4vec": jet_arr}
        data_dict["weight_mc_combined"] = _weights * _scale_factor
        fill_or_extend_tree(df_test, data_dict)
    # put what's needed into test and ignore the rest
    elif current_index < max_test_index and current_index + current_data_size >= max_test_index:
        split_index = max_test_index - current_index
        data_dict = {"jet_4vec": jet_arr[:split_index]}
        data_dict["weight_mc_combined"] = _weights[:split_index] * _scale_factor
        fill_or_extend_tree(df_test, data_dict)
    else:
        print("Uncaught case:",
              "current_index: {}".format(current_index),
              "current_data_size: {}".format(current_data_size),
              "max_train_index: {}".format(max_train_index),
              "max_val_index: {}".format(max_val_index),
              "max_test_index: {}".format(max_test_index), sep='\n')
    
    current_index += current_data_size
    if current_index >= max_test_index:
        break
print("Finished!")

Loading file: /scratch/mdd424/CARL_tthbb/data/Powheg/semilep/user.aknue.mc16_13TeV.600791.PhPy8EG_NNPDF31ME_ttbb_4FS_pTdef1_dilep.TOPQ1.e8332s3126r10201p4696.TTHbb212206-v1n_1l_out_root/user.aknue.28425208._000008.1l_out.root
Loading file: /scratch/mdd424/CARL_tthbb/data/Powheg/semilep/user.aknue.mc16_13TeV.600792.PhPy8EG_NNPDF31ME_ttbb_4FS_pTdef1_ljets.TOPQ1.e8332s3126r10724p4696.TTHbb212206-v1n_1l_out_root/user.aknue.28425062._000025.1l_out.root
Loading file: /scratch/mdd424/CARL_tthbb/data/Powheg/semilep/user.aknue.mc16_13TeV.600792.PhPy8EG_NNPDF31ME_ttbb_4FS_pTdef1_ljets.TOPQ1.e8332s3126r9364p4696.TTHbb212206-v1n_1l_out_root/user.aknue.28425556._000005.1l_out.root
Loading file: /scratch/mdd424/CARL_tthbb/data/Powheg/semilep/user.aknue.mc16_13TeV.600792.PhPy8EG_NNPDF31ME_ttbb_4FS_pTdef1_ljets.TOPQ1.e8332s3126r10201p4696.TTHbb212206-v1n_1l_out_root/user.aknue.28425211._000003.1l_out.root
Loading file: /scratch/mdd424/CARL_tthbb/data/Powheg/semilep/user.aknue.mc16_13TeV.600792.PhPy8EG

In [17]:
df_train = uproot.recreate("mixture_model_data/Sherpa_ttbb_full_training_DeepSets_data.root")
df_val = uproot.recreate("mixture_model_data/Sherpa_ttbb_full_validation_DeepSets_data.root")
df_test = uproot.recreate("mixture_model_data/Sherpa_ttbb_full_testing_DeepSets_data.root")

current_index = 0
for filename in all_variation_files:
    print("Loading file: {}".format(filename))
    # Load the data
    variation_dataset = uproot.open(filename)["nominal_Loose"].arrays(jet_features + weight_features)
    # Get the run number
    variation_file_run_number = str(VARIATION_FILE_TO_RUN_NUMBER[filename])
    # Get the DSID number
    variation_file_dsid = str(VARIATION_FILE_TO_DSID[filename])
    # Get the luminsotiy, DSID cross section, and per DSID total weighted events
    _scale_factor = luminosities_by_run[variation_file_run_number] * VARIATION_XSECTIONS[variation_file_dsid] / VARIATION_NORMALIZATIONS[variation_file_dsid]
    # Extract the combined weight array
    _weights = ak.concatenate(ak.unzip(variation_dataset[weight_features][:, np.newaxis]), axis=1).to_numpy().prod(axis=1)
    
    current_data_size = len(_weights)
    
    jet_sets = []
    for i in range(current_data_size):
        jet_sets.append(
            np.concatenate([x.to_numpy()[:, np.newaxis] for x in ak.unzip(variation_dataset[jet_features][i])], axis=1).flatten()
        )
    jet_arr = ak.Array(jet_sets)
    
    # put everything in train
    if current_index + current_data_size < max_train_index:
        data_dict = {"jet_4vec": jet_arr}
        data_dict["weight_mc_combined"] = _weights * _scale_factor
        fill_or_extend_tree(df_train, data_dict)
    # put part in train and the rest in val
    elif current_index < max_train_index and current_index + current_data_size < max_val_index:
        split_index = max_train_index - current_index
        data_dict = {"jet_4vec": jet_arr[:split_index]}
        data_dict["weight_mc_combined"] = _weights[:split_index] * _scale_factor
        fill_or_extend_tree(df_train, data_dict)
        
        data_dict = {"jet_4vec": jet_arr[split_index:]}
        data_dict["weight_mc_combined"] = _weights[split_index:] * _scale_factor
        fill_or_extend_tree(df_val, data_dict)
    # put everything into val
    elif current_index >= max_train_index and current_index + current_data_size < max_val_index:
        data_dict = {"jet_4vec": jet_arr}
        data_dict["weight_mc_combined"] = _weights * _scale_factor
        fill_or_extend_tree(df_val, data_dict)
    # put part in val and the rest in test
    elif current_index < max_val_index and current_index + current_data_size < max_test_index:
        split_index = max_val_index - current_index
        data_dict = {"jet_4vec": jet_arr[:split_index]}
        data_dict["weight_mc_combined"] = _weights[:split_index] * _scale_factor
        #fill_or_extend_tree(df_val, data_dict)
        
        data_dict = {"jet_4vec": jet_arr[split_index:]}
        data_dict["weight_mc_combined"] = _weights[split_index:] * _scale_factor
        fill_or_extend_tree(df_test, data_dict)
    # put everything into test
    elif current_index >= max_val_index and current_index + current_data_size < max_test_index:
        data_dict = {"jet_4vec": jet_arr}
        data_dict["weight_mc_combined"] = _weights * _scale_factor
        fill_or_extend_tree(df_test, data_dict)
    # put what's needed into test and ignore the rest
    elif current_index < max_test_index and current_index + current_data_size >= max_test_index:
        split_index = max_test_index - current_index
        data_dict = {"jet_4vec": jet_arr[:split_index]}
        data_dict["weight_mc_combined"] = _weights[:split_index] * _scale_factor
        fill_or_extend_tree(df_test, data_dict)
    else:
        print("Uncaught case:",
              "current_index: {}".format(current_index),
              "current_data_size: {}".format(current_data_size),
              "max_train_index: {}".format(max_train_index),
              "max_val_index: {}".format(max_val_index),
              "max_test_index: {}".format(max_test_index), sep='\n')
    
    current_index += current_data_size
    if current_index >= max_test_index:
        break
print("Finished!")

Loading file: /scratch/mdd424/CARL_tthbb/data/Sherpa/semilep/user.alheld.mc16_13TeV.700165.Sh_2210_ttbb_ljetsP.TOPQ1.e8263a875r10724p4434.TTHbb212206-v1n_1l_out_root/user.alheld.28403455._000023.1l_out.root
Loading file: /scratch/mdd424/CARL_tthbb/data/Sherpa/semilep/user.alheld.mc16_13TeV.700166.Sh_2210_ttbbljetsM.TOPQ1.e8263a875r10201p4434.TTHbb212206-v1n_1l_out_root/user.alheld.28403761._000039.1l_out.root
Loading file: /scratch/mdd424/CARL_tthbb/data/Sherpa/semilep/user.alheld.mc16_13TeV.700166.Sh_2210_ttbbljetsM.TOPQ1.e8263a875r10724p4434.TTHbb212206-v1n_1l_out_root/user.alheld.28403457._000023.1l_out.root
Loading file: /scratch/mdd424/CARL_tthbb/data/Sherpa/semilep/user.alheld.mc16_13TeV.700166.Sh_2210_ttbbljetsM.TOPQ1.e8263a875r10201p4434.TTHbb212206-v1n_1l_out_root/user.alheld.28403761._000007.1l_out.root
Loading file: /scratch/mdd424/CARL_tthbb/data/Sherpa/semilep/user.alheld.mc16_13TeV.700166.Sh_2210_ttbbljetsM.TOPQ1.e8263a875r10724p4434.TTHbb212206-v1n_1l_out_root/user.alhel

In [138]:
data_dict

{'jet_4vec': <Array [[[1.13e+05, -0.753, ... 8.41e+04]]] type='244098 * var * var * float64'>,
 'weight_mc_combined': array([ 0.00393443,  0.00015745,  0.00444214, ..., -0.00029049,
         0.00704612,  0.00770962], dtype=float32)}

### Testing

In [45]:
ak.concatenate(ak.unzip(nominal_dataset[jet_features][0]), axis=-1).to_numpy().shape

(20,)

In [35]:
nominal_dataset["jet_pt"][0].to_numpy()

array([184193.02 ,  78562.62 ,  55625.71 ,  54132.633,  37286.516],
      dtype=float32)

In [44]:
ak.unzip(nominal_dataset[jet_features][0])

(<Array [1.84e+05, 7.86e+04, ... 3.73e+04] type='5 * float32'>,
 <Array [-0.621, -0.621, ... 2.44, -0.555] type='5 * float32'>,
 <Array [2.39, -1.75, -2.19, 2.5, 1.01] type='5 * float32'>,
 <Array [2.21e+05, 9.44e+04, ... 4.37e+04] type='5 * float32'>)

In [51]:
np.asarray(ak.concatenate(ak.unzip(nominal_dataset[jet_features][:, np.newaxis][0]), axis=1))

(1, 20)

In [64]:
ak.concatenate([x[:, np.newaxis] for x in ak.unzip(nominal_dataset[jet_features][0])], axis=1).type

5 * var * float32

In [62]:
ak.unzip(nominal_dataset[jet_features][1])[0].to_numpy()

array([127191.836, 100057.266,  64308.777,  54186.504,  26848.889,
        26609.314], dtype=float32)

<Array [[1.28e+05, -1.99, ... 9.46e+04]] type='8 * 4 * ?float32'>

In [116]:
jet_sets = []
for i in range(5):
    jet_sets.append(
        ak.concatenate(
            [x[:, np.newaxis] for x in ak.unzip(nominal_dataset[jet_features][i])],
            axis=1
        )
    )
jet_arr = ak.Array(jet_sets)

In [117]:
for x in jet_arr:
    print(x.to_numpy().shape)

(5, 4)
(6, 4)
(4, 4)
(4, 4)
(8, 4)


In [115]:
ak.concatenate(ak.unzip(nominal_dataset[jet_features][0][np.newaxis]), axis=0).to_numpy().T[:, 0]

array([184193.02 ,  78562.62 ,  55625.71 ,  54132.633,  37286.516],
      dtype=float32)

In [113]:
nominal_dataset["jet_pt"][0].to_numpy()

array([184193.02 ,  78562.62 ,  55625.71 ,  54132.633,  37286.516],
      dtype=float32)

In [118]:
jet_arr

<Array [[[1.84e+05, -0.621, ... 9.46e+04]]] type='5 * var * var * float64'>

In [119]:
dir(jet_arr)

['Mask',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__self__',
 '__self_class__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__thisclass__',
 '_behavior',
 '_caches',
 '_layout',
 '_numbaview',
 'behavior',
 'caches',
 'fields',
 'layout',
 'mask',
 'nbytes',
 'ndim',
 'numba_type',
 'slot0',
 'slot1',
 'slot2',
 'slot3',
 'slot4',
 'slot5',
 'slot6',
 'slot7',
 'slot8',
 'slot9',
 'to_list',
 'to_numpy',
 'tolist',
 'type']

In [122]:
%timeit ak.concatenate(ak.unzip(nominal_dataset[jet_features][0][np.newaxis]), axis=0).to_numpy().T

7.48 ms ± 60.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [127]:
%timeit ak.concatenate([x[:, np.newaxis] for x in ak.unzip(nominal_dataset[jet_features][0])], axis=1)

754 µs ± 1.03 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [139]:
%timeit np.concatenate([x.to_numpy()[:, np.newaxis] for x in ak.unzip(nominal_dataset[jet_features][0])], axis=1).flatten()

362 µs ± 3.51 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [131]:
%timeit np.concatenate([np.asarray(x)[:, np.newaxis] for x in ak.unzip(nominal_dataset[jet_features][0])], axis=1)

550 µs ± 2.28 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [145]:
jet_sets = []
for i in range(8):
    jet_sets.append(
        np.concatenate([x.to_numpy()[:, np.newaxis] for x in ak.unzip(nominal_dataset[jet_features][i])], axis=1).flatten()
    )
jet_arr = ak.Array(jet_sets)
jet_arr

<Array [[1.13e+05, -0.753, ... 3.22e+04]] type='8 * var * float64'>

In [146]:
for arr in jet_arr:
    print(arr.type)

16 * float64
16 * float64
16 * float64
16 * float64
24 * float64
16 * float64
24 * float64
20 * float64
